In [ ]:
import requests 
import json 
import urllib
import re
root_dir = 'D:\scrapyfile'
'''
爬取糗事百科中图片
'''

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

url = 'https://pic.qiushibaike.com/system/pictures/12415/124156185/medium/S0QSW2LLWZHZ7N8F.jpg'
'''
content 二进制
json 对象
text 字符串
'''
img_data = requests.get(url=url).content

with open('D:\scrapyfile\qiutu.jpg', 'wb') as f: 
    f.write(img_data)



In [ ]:
img_url = 'https://pic.qiushibaike.com/system/pictures/12415/124156185/medium/S0QSW2LLWZHZ7N8F.jpg'
urllib.request.urlretrieve(img_url, 'D:\scrapyfile\qiutu.jpg')

* 使用urllib的方式爬取图片 无法进行UA封装，而requests的方式可以


* 如果在进行数据解析的时候，一定是需要对页面布局进行分析 如果当前网站没有动态加载的数据就可以直接使用Elements对页面布局进行分析，否则只可以使用networks对页面数据进行分析


In [ ]:
# 捕获当前首页的页面源码数据
url = 'http://www.521609.com/tuku/mxxz/'
page_text = requests.get(url = url, headers = headers).text

In [ ]:
# <li style="position: absolute; left: 0px; top: 0px;"> 
#     <a href="/tuku/1221.html" title="Angelababy变身复古少女漫步古堡 车厘子红唇妆显别样风情">
#         <img src="/d/file/p/2021/03-09/6e7addfae4a6507b80bfc3f853a6ea3b.jpg" alt="Angelababy变身复古少女漫步古堡 车厘子红唇妆显别样风情">
#         <p>Angelababy变身复古少女漫步古堡 车厘子红唇妆显别样风情</p>
#     </a>
# </li>

# 正则表达式教程
[https://www.runoob.com/regexp/regexp-syntax.html](参考链接)


In [ ]:
import os 
import bs4
dirName = 'D:\scrapyfile\ImgLibs'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    
ex = '<img src="(.*)" alt=.*>'
img_src_list = re.findall(ex, page_text) # re.S 处理换行
for src in img_src_list:
    src = 'http://www.521609.com/' + src 
    imgPath = dirName + '/' + src.split('/')[-1]
    urllib.request.urlretrieve(src, imgPath)
    print("下载成功")

# BS4

* 数据解析的原理：指定标签的定位
* 数据解析的原理：取出标签中存储的数据或者标签属性中的数据

- BeautifulSoup对象的实例化
    - BeautifulSoup(fp, 'lxml') 用来将本地存储的html文档中的数据进行解析
    - BeautifulSoup(page_text, 'lxml') 用来将互联网上请求到的页面源码数据进行解析
   
- 标签定位：
    - soup.tagName: 只可以定位到第一次出现的tagName标签
    - soup.find('tagName', attrName='value') 属性定位
    - soup.findAll: 跟find一样用作属性定位，只不过findAll返回的是列表
    - soup.select() 选择器
        - 类选择器
        - id选择器
        - 层级选择器
            - >:表示一个层级
            - 空格: 表示多个层级
            
- 取数据:
    - .text 返回的是该标签下所有的文本内容
    - .string 返回的是该标签直系的文本内容
        

# BS4实战

爬取三篇全篇内容：[三国演义](https://www.shicimingju.com/book/sanguoyanyi.html)

In [ ]:
main_url = 'https://www.shicimingju.com/book/sanguoyanyi.html'
page_url = requests.get(url=main_url, headers=headers).text
from bs4 import BeautifulSoup
# 数据解析： 章节标题、详情url、章节内容
html = urlopen(main_url)
soup = BeautifulSoup(html)
a_list = soup.select('.book-mulu > ul > li > a')
print(len(a_list))
fp = open('D:\scrapyfile\sanguo.txt','w',encoding='utf-8')
for a in a_list:
    title = a.string 
    detail_url = 'https://www.shicimingju.com/' + a['href']
    html = urlopen(detail_url)
    soup = BeautifulSoup(html)
    content = soup.find('div', class_='chapter_content').text
    fp.write(title+":"+content+"\n")
    print(title+"\t 已经保存完毕")
fp.close()

# xpath
- 环境安装
    - pip install lxml
- 解析原理： html标签是以树状形式进行展示
    - 1.实例化一个etree的对象，且将待解析的页面源码数据加载到该对象中
    - 2.调用etree对象的xpath方法，结合着不同的xpath表达式实现标签的定位和数据提取
- 实例化etree对象
    - etree.parse('fileName'):将本地html文档加载到该对象中
    - etree.HTML('page_text'):网站获取的页面数据加载到该对象中
- 标签定位
    - 最左边/: 如果xpath表达式最左侧是以/开头则表示该表达式一定要从根标签开始定位指定标签
    - 非最左侧的/:表示一个层级
    - 非最左侧的//： 表示多个层级
    - 最左侧的//: xpath表达式可以从任意位置进行标签定位
    - 属性定位: tagName[@attrName="value"]
    - 索引定位: tag[index] 索引从1开始的
    - 模糊匹配:
        - //div[contains(@class, "ng")]
        - //div[starts-with(@class, "ta")]
- 获取文本
    - /text(): 直系文本内容
    - //text():所有的文本内容
- 获取属性
    - /@attrName 
- 局部数据解析：
    - 我们要将定位到的页面中的标签作为待解析的数据，再次使用xpath表达式解析待解析的数据
    - 在局部数据解析的时候，xpath表达式中要使用./的操作，./表示的就是当前的局部数据

In [ ]:
from lxml import etree
img_dir = 'D:/scrapyfile/img1'
if not os.path.exists(img_dir):
    os.mkdir(img_dir)
url = 'https://pic.netbian.com/4kmeinv/'
response  = requests.get(url=url, headers=headers)
response.encoding = 'gbk'
page_text = response.text
tree = etree.HTML(page_text)
li_list = tree.xpath('//div[@class="slist"]/ul/li')
for li in li_list:
#     print(type(li)) li的数据类型和tree的数据类型一样，li也可以调用xpath的方法
    title = li.xpath('./a/img/@alt')[0] + '.jpg'
    img_src = 'https://pic.netbian.com' + li.xpath('./a/img/@src')[0]
    img_data = requests.get(url=img_src, headers=headers).content
    img_path = img_dir + '/' + title
    with open(img_path, 'wb') as fp:
        fp.write(img_data)
    fp.close()

In [ ]:
# 分页爬取
img_dir = 'D:/scrapyfile/img2'
if not os.path.exists(img_dir):
    os.mkdir(img_dir)
url = 'https://pic.netbian.com/4kmeinv/index_%d.html'
for page in range(1,6):
    if page == 1:
        new_page = 'https://pic.netbian.com/4kmeinv'
    else:   
        new_page = format(url % page)
    response = requests.get(url = new_page, headers = headers)
    response.encoding = 'gbk'
    page_text = response.text
    tree = etree.HTML(page_text)
    li_list = tree.xpath('//div[@class="slist"]/ul/li')
    for li in li_list:
        title = li.xpath('./a/img/@alt')[0] + '.jpg'
        img_src = 'https://pic.netbian.com' + li.xpath('./a/img/@src')[0]
        img_data = requests.get(url=img_src, headers=headers).content
        img_path = img_dir + '/' + title
        with open(img_path, 'wb') as fp:
            fp.write(img_data)
        fp.close()

In [ ]:
soup.a

In [ ]:
# 将https://www.aqistudy.cn/historydata/ 所有城市名称解析出来
import requests 
from lxml import etree
from bs4 import BeautifulSoup
url = 'https://www.aqistudy.cn/historydata/'
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}
page_text = requests.get(url = url, headers=headers).text
tree = etree.HTML(page_text)
hot_cities = tree.xpath('//div[@class="bottom"]/ul/li/a/text()')
all_cities = tree.xpath('//div[@class="bottom"]/ul/div[2]/li/a/text()')
# tree.xpath('//div[@class="bottom"]/ul/li/a/text() | //div[@class="bottom"]/ul/div[2]/li/a/text()')

In [ ]:
# 实战一： 站长素材 下载高清图片
import requests 
from lxml import etree
from bs4 import BeautifulSoup
import os 

dirName = 'D:\scrapyfile\shuaiqi'

if not os.path.exists(dirName):
    os.mkdir(dirName)
    
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}
url_template = 'https://sc.chinaz.com/tag_tupian/baqi_%d.html' 

for page in range(1,6):
    if page == 1:
        url = 'https://sc.chinaz.com/tag_tupian/baqi.html'
    else:
        url = format(url_template%page)
        

    response = requests.get(url = url, headers = headers)
    # 一定要注意编码 出现乱码的时候 就要尝试切换编码方式
    response.encoding = 'utf-8'
    page_text = response.text
    tree = etree.HTML(page_text)
    img_list = tree.xpath('//div[@id="container"]/div/div/a/img')
    '''
    涉及到图片懒加载，可视化区域才加载，这时候要爬取的是伪属性
    '''

    for img in img_list:
        new_url = 'https:' + img.xpath('./@src2')[0]
        img_title = img.xpath('./@alt')[0]
#         print(new_url)
        img_data = requests.get(url = new_url, headers = headers).content
        with open(dirName+'\\' + img_title + '.jpg', 'wb') as f:
            f.write(img_data)
    f.close()
    
    print(str(page)+'\t over')
    

In [ ]:
# 实战二： 站长素材 下载简历
import requests 
from lxml import etree
from bs4 import BeautifulSoup
import os 

dirName = 'D:/scrapyfile/jianli'

if not os.path.exists(dirName):
    os.mkdir(dirName)
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}
url = 'https://sc.chinaz.com/jianli/tongyong.html'
response = requests.get(url = url, headers = headers)
response.encoding = 'utf-8'
page_text = response.text
tree = etree.HTML(page_text)
img_list = tree.xpath('//div[@id="main"]/div/div/a')
for img in img_list:
    page_url = img.xpath('./@href')
    img_title = img.xpath('./img/@alt')
    img_url = "https:" + img.xpath('./img/@src')[0]

In [ ]:
# 实战三： 站长素材 脚本下载
import requests 
from lxml import etree
from bs4 import BeautifulSoup
import os 

dirName = 'D:/scrapyfile/jiaoben'

if not os.path.exists(dirName):
    os.mkdir(dirName)

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

url_template = 'https://sc.chinaz.com/jiaoben/index_%d.html'
for page in range(1,6):
    if page == 1:
        url = 'https://sc.chinaz.com/jiaoben/index.html'
    else:
        url = format(url_template % page) 
    response = requests.get(url = url, headers = headers)
    response.encoding = 'utf-8'
    page_text = response.text
    tree = etree.HTML(page_text)
    jl_list = tree.xpath('//div[@id="container"]/div/div[2]/a')
    for jl in jl_list:
        new_page = 'https:' + jl.xpath('./@href')[0]
        title = jl.xpath('./@alt')[0]
    #     print(title,'\t',  new_page) 
        new_response = requests.get(url = new_page, headers = headers)
        new_response.encoding = 'utf-8'
        new_text = etree.HTML(new_response.text)
        file_url = new_text.xpath('//div[@class="dian"]/a/@href')[0]
        file = requests.get(url = file_url, headers = headers).content
        file_name = dirName + '/' + title + '.rar'
        with open(file_name, 'wb') as f:
            f.write(file)
        f.close()
    print(str(page) + '\t over')